In [1]:
#profitableHospital-project

In [2]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [3]:
# ... File Imports

In [4]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2017.csv")
df.head(2)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (105) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,4492364.0,2638799.0,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,31174.0,-1114660.0,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095


In [5]:
#Quick Check
df['Provider CCN'].nunique()

6075

In [6]:
#Quick Check
df['Provider CCN'].count()

6171

In [7]:
#6367-6171=196
#6171-6075=96

In [8]:
df.drop_duplicates(inplace = True)
df['Provider CCN'].count()
#df.count()

6171

In [9]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [10]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [11]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("geocoded2017.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,510013,39.946236,-80.752966
1,340106,34.901612,-79.708561


In [12]:
#Quick Check
geoTable['Provider CCN'].count()

6171

In [13]:
geoTable.drop_duplicates(inplace = True)
geoTable['Provider CCN'].count()
#df.count()

6076

In [14]:
#6171-6076=95

In [15]:
# ... Appending of source files into one main working file

In [16]:
mainFile1 = df

In [17]:
#Append a lattitude & longitude
mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561


In [18]:
#Quick Check
mainFile1['Provider CCN'].count()

6173

In [19]:
mainFile1.drop_duplicates(inplace = True)
mainFile1['Provider CCN'].count()
#df.count()

6173

In [20]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])
#mainFile1.head()

In [21]:
#Quick Check
mainFile1['Provider CCN'].count()

6173

In [22]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [23]:
#Quick Check
mainFile1['Provider CCN'].count()

6173

In [24]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng,CON,Urban
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966,1.0,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561,1.0,0.0


In [25]:
mainFile1.to_csv("main1_2017.csv", index=False, header=True)

In [26]:
#mainFile1['Provider CCN'].duplicated()

In [27]:
mainFile1[mainFile1['Provider CCN'].duplicated(keep = False)].head(200)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng,CON,Urban
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966,1.0,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561,1.0,0.0
3,341317,PIONEER HLTH SERV OF STOKES CNTY,1570 NC & 89 HWY,DANBURY,NC,27016,STOKES,49180.0,U,CAH,...,1.154767,578366.0,801800.0,159516.0,702117.0,601721,36.418001,-80.230915,1.0,1.0
4,110003,MAYO CLINIC HEALTH SYSTEM WAYCROSS,1900 TEBEAU STREET,WAYCROSS,GA,31501,WARE,99911.0,R,STH,...,0.271734,3113423.0,11231324.0,1331.0,8211.0,603846,31.224905,-82.347732,1.0,0.0
5,250025,GILMORE MEMORIAL REGIONAL MEDICAL CE,1105 EARL FRYE BLVD,AMORY,MS,38821,MONROE,99925.0,R,STH,...,0.152035,2548541.0,17164229.0,NaN,NaN,604167,33.976509,-88.476737,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,360055,TRUMBULL MEMORIAL HOSPITAL,1350 EAST MARKET ST.,WARREN,OH,44483,NaN,49660.0,U,STH,...,0.150316,10197748.0,114247007.0,NaN,NaN,670074,41.235825,-80.798391,1.0,1.0
6062,360055,TRUMBULL MEMORIAL HOSPITAL,1350 EAST MARKET ST.,WARREN,OH,44483-,TRUMBULL,49660.0,U,STH,...,0.126383,5581156.0,59582432.0,NaN,NaN,670360,41.235825,-80.798391,1.0,1.0
6082,452046,PLAZA SPECIALTY HOSPITAL,1300 BINZ STREET,HOUSTON,TX,77004,NaN,26420.0,U,LTCH,...,NaN,NaN,NaN,NaN,NaN,670422,29.723504,-95.386924,0.0,1.0
6127,54093,LLU BEHAVIORAL MEDICINE CENTER,1710 BARTON ROAD,REDLANDS,CA,92373,SAN BERNARDINO,40140.0,U,PH,...,NaN,NaN,NaN,NaN,NaN,670626,34.049358,-117.214026,0.0,1.0


In [30]:
mainFile1['Provider CCN'].drop_duplicates(inplace = True)
mainFile1['Provider CCN'].count()
#df.count()

6173

In [31]:
# zz = mainFile1.sort_values(by= 'Provider CCN')
# zz[zz['Provider CCN'] == 510013].head(100)

In [32]:
# ... Add calculated fields

In [33]:
mainFile2 = mainFile1

In [34]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [35]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [36]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [37]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [38]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [39]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge
#mainFile2.head()

In [40]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed
#mainFile2.head()

In [41]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed
#mainFile2.head()

In [42]:
#Append calculated field revs per fte
# revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
# mainFile2["Rev Per FTE"] = revfte
#mainFile2.head()

In [43]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed
#mainFile2.head()

In [44]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays
#mainFile2.head()

In [45]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed
#mainFile2.head()

In [46]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent
#wagepercent.head()

In [47]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent
#overheadpercent.head()

In [48]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent
#overheadpercent.head()

In [49]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte
#wagepercent.head()

In [50]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed
#wagepercent.head()

In [51]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit
#mainFile2.head()

In [52]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]
#netinc.head()

In [56]:
# Box plot and outlier calculations for outlier approach
netinc2 = netinc.dropna(how='any')
chartdata = netinc2
# fig1, ax1 = plt.subplots()
# ax1.set_title('Net Income Percent')
# ax1.set_ylabel('Net Income Percent')
# ax1.boxplot(chartdata)
# plt.show()

In [57]:
#Quartile calculations
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: -0.03084132008127647
The upper quartile of profit percent is: 0.11374176480973226
The interquartile range of profit percent is: 0.14458308489100874
The the median of profit percent is: 0.038000703630347375 
Values below -0.24771594741778957 could be outliers.
Values above 0.3306163921462454 could be outliers.


In [58]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [59]:
#Quintile attempt
# quartiles = chartdata.quantile([.2,.5,.8])
# lowerq = quartiles[0.2]
# upperq = quartiles[0.8]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [60]:
#Create a bronze, silver, gold score for clasifying net income performance
# bins = [-100, -0.0542, .03804, 0.13654, 100]
# group_names = ["4","3", "2", "1"]

In [61]:
#Add a bronze, silver, gold scoring system to the dataset
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [62]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    2945
3    1475
1    1473
Name: Net Income Score, dtype: int64

In [63]:
#Quick Check
mainFile2['Provider CCN'].count()

6173

In [64]:
mainFile2.to_csv("main2_2017.csv", index=False, header=True)

In [65]:
# ..... File targeting

In [66]:
mainFile3 = mainFile2
mainFile3['Provider CCN'].count()

6173

In [67]:
#Targets dataset on Colorad0
#mainFile3 = mainFile3.loc[df['State Code'] == 'CO']

In [71]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'PR']

In [72]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'GU']

In [73]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'VI']

In [74]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'MP']

In [75]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'HI']

In [76]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'AK']

In [78]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6056

In [81]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile3['Net Patient Revenue'] >= 0.1]

In [82]:
#Quick Check
mainFile3['Provider CCN'].count()

5783

In [83]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [84]:
#Quick Check
mainFile3['Provider CCN'].count()

5734

In [85]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile2['FTE - Employees on Payroll'] >= 10]

In [86]:
#Quick Check
mainFile3['Provider CCN'].count()

5711

In [87]:
#Targets dataset on certificate of need states
#mainFile3 = mainFile3.loc[mainFile2['CON'] == 1]

In [88]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] <= .3304]

In [89]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] >= -.2473]

In [90]:
#Remove Discharges Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 90000]

In [91]:
#Remove Rev Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Bed'] <= 2000000]

In [92]:
#Remove Rev Per FTE outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per FTE'] <= 500000]

In [93]:
#Remove Rev Per Discharge outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 175000]

In [94]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
mainFile3 = mainFile3.loc[mainFile2['CCN Facility Type'] == 'STH']

In [95]:
#Quick Check
mainFile3['Provider CCN'].count()

3194

In [96]:
#Quick check on distribution of bronze, silver and gold
#mainFile3['Net Income Score'].value_counts()

In [97]:
#Quick Check
mainFile3['Provider CCN'].count()

3194

In [98]:
#mainFile3.to_csv("main3_2017.csv", index=False, header=True)

In [99]:
# ... Target file based on relevant features

In [100]:
mainFile4 = mainFile3

In [101]:
#Targeting the dataset on metrics important to Kent
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,...,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,U,STH,10/01/2016 12:00:00 AM,12/31/2016 12:00:00 AM,...,6.214286,0.067547,6.214286,0.308395,1,0.715116,0.627406,0.031451,18996.972990,76691.942857
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,R,STH,10/01/2016 12:00:00 AM,11/30/2016 12:00:00 AM,...,3.062500,0.050205,3.062500,-0.337043,3,0.509928,0.924882,0.073496,61909.838473,47792.984375


In [103]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    3194
Hospital Name                                   3194
Street Address                                  3193
City                                            3194
State Code                                      3194
Zip Code                                        3194
Rural Versus Urban                              3194
CCN Facility Type                               3194
Fiscal Year Begin Date                          3194
Fiscal Year End Date                            3194
FTE - Employees on Payroll                      3194
Number of Beds                                  3194
Total Bed Days Available                        3194
Total Discharges (V + XVIII + XIX + Unknown)    3194
Net Patient Revenue                             3194
Net Income                                      3193
Wage-Related Costs (Core)                       3183
Total Salaries (adjusted)                       3184
CON                                           

In [104]:
mainFile4 = mainFile4.dropna(how='any')

In [105]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2982
Hospital Name                                   2982
Street Address                                  2982
City                                            2982
State Code                                      2982
Zip Code                                        2982
Rural Versus Urban                              2982
CCN Facility Type                               2982
Fiscal Year Begin Date                          2982
Fiscal Year End Date                            2982
FTE - Employees on Payroll                      2982
Number of Beds                                  2982
Total Bed Days Available                        2982
Total Discharges (V + XVIII + XIX + Unknown)    2982
Net Patient Revenue                             2982
Net Income                                      2982
Wage-Related Costs (Core)                       2982
Total Salaries (adjusted)                       2982
CON                                           

In [106]:
#Output targeted file
mainFile4.to_csv("main4_2017.csv", index=False, header=True)

In [107]:
# ... Net Income to feature review

In [108]:
targetFile5 = mainFile4

In [109]:
#Outlier Review
chartdata = targetFile5["Discharges Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 32.78753791708797
The upper quartile of profit percent is: 55.0977833485895
The interquartile range of profit percent is: 22.310245431501535
The the median of profit percent is: 46.09932011795544 
Values below -0.6778302301643393 could be outliers.
Values above 88.56315149584181 could be outliers.


In [110]:
#NI v. discharges per bed
# x_axis = targetFile5["Discharges Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed")
# plt.xlabel("Discharges Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [111]:
#NI v. discharges per bed days
# x_axis = targetFile5["Discharges Per Bed Days"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed Days")
# plt.xlabel("Discharges Per Bed Days")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# #ax.set_yscale('log')
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [112]:
#Outlier Review
chartdata = targetFile5["Rev Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 804362.7826709477
The upper quartile of profit percent is: 1590219.5808389264
The interquartile range of profit percent is: 785856.7981679786
The the median of profit percent is: 1154970.1033051123 
Values below -374422.41458102036 could be outliers.
Values above 2769004.778090894 could be outliers.


In [113]:
#NI v. Revenue Per Bed
# x_axis = targetFile5["Rev Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Bed")
# plt.xlabel("Rev Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [115]:
#Outlier Review
# chartdata = targetFile5["Rev Per FTE"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [116]:
#NI v. Rev Per FTE
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Rev Per FTE'] <= 1000000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Rev Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Rev Per FTE")
# plt.xlabel("Rev Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [118]:
#Outlier Review
chartdata = targetFile5["Rev/Discharge"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 20249.235054374352
The upper quartile of profit percent is: 34700.51973767634
The interquartile range of profit percent is: 14451.284683301987
The the median of profit percent is: 25606.19220656314 
Values below -1427.6919705786277 could be outliers.
Values above 56377.44676262932 could be outliers.


In [119]:
#NI v. Rev Per Discharge
# x_axis = targetFile5["Rev Per Discharge"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Discharge")
# plt.xlabel("Rev Per Discharge")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [120]:
#Outlier Review
chartdata = targetFile5["LbrExp/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 75888.99972993067
The upper quartile is: 96877.119655674
The interquartile range is: 20988.119925743333
The the median of is: 85290.95879189257 
Values below 44406.81984131567 could be outliers.
Values above 128359.29954428901 could be outliers.


In [121]:
#NI v. Labor Expense Per FTE
# targetFile6 = targetFile5.loc[targetFile5['Labor Exp Per FTE'] <= 130000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor Exp Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor Exp Per FTE")
# plt.xlabel("Labor Exp Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [122]:
#Outlier Review
chartdata = targetFile5["Overhead %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.5621077646917462
The upper quartile is: 0.6926348487601393
The interquartile range is: 0.13052708406839308
The the median of is: 0.6208035039207582 
Values below 0.36631713858915654 could be outliers.
Values above 0.8884254748627289 could be outliers.


In [123]:
#NI v. Overhead Percent
# targetFile6 = targetFile5.loc[targetFile5['Overhead %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Overhead %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Overhead %")
# plt.xlabel("Overhead %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [124]:
#Outlier Review
chartdata = targetFile5["Labor %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.3751075906483552
The upper quartile is: 0.5373285012672162
The interquartile range is: 0.16222091061886101
The the median of is: 0.45013593995322915 
Values below 0.13177622472006367 could be outliers.
Values above 0.7806598671955077 could be outliers.


In [125]:
#NI v. Labor Percent
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Labor %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor %")
# plt.xlabel("Labor %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [126]:
#Outlier Review
chartdata = targetFile5["Discharge/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 32.78753791708797
The upper quartile is: 55.0977833485895
The interquartile range is: 22.310245431501535
The the median of is: 46.09932011795544 
Values below -0.6778302301643393 could be outliers.
Values above 88.56315149584181 could be outliers.


In [127]:
#NI v. Labor Percent
# targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Discharges Per FTE'] <= 120]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Discharges Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Discharges Per FTE")
# plt.xlabel("Discharges Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [128]:
#Start machine learning ...

In [129]:
# Select features to be used as x values
features1 = mainFile4[['CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
features1.head(2)
#RFE logrithmic output ... array([15, 17, 16, 14, 12,  8, 11,  6,  5 (R/BDay),  7,  9,  3(D/FTE),  2(L%),  1 (OH%),  4 (D%), 13, 10])

,CON,Urban,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,1.0,1.0,0.311881,0.688119,0.003080,0.003080,19670.239080,122236.485714,1328.657453,6.214286,0.067547,6.214286,0.715116,0.627406,0.031451,18996.972990,76691.942857
1,1.0,0.0,0.325730,0.674270,0.001075,0.001075,16873.362245,51674.671875,847.125768,3.062500,0.050205,3.062500,0.509928,0.924882,0.073496,61909.838473,47792.984375


In [130]:
#Output features file as a reference document
features1.to_csv("features2017.csv", index=False, header=True)

In [131]:
# ... Linear Regression Model

In [132]:
# Identify the outcome column/data
y = mainFile4[['Net Income %']]
y.head()

,Net Income %
0,0.308395
1,-0.337043
5,-0.132426
6,-0.291462
7,-0.050419


In [133]:
mainFile4["Net Income %"].dtype

dtype('float64')

In [134]:
y.shape

(2982, 1)

In [135]:
X = features1
X.shape

(2982, 17)

In [136]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [137]:
y_train.shape

(2236, 1)

In [138]:
X_train.shape

(2236, 17)

In [139]:
y_test.shape

(746, 1)

In [140]:
X_test.shape

(746, 17)

In [141]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [142]:
#Create the linear regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LinearRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.38644176850279865


In [143]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.5476352023552173


In [144]:
# ... Linear Recursive Feature Elimination

In [145]:
#Identify features to create a more efficient model with
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_scaled, y_train)

In [146]:
selector.support_

array([False, False,  True,  True,  True, False, False, False, False,
        True, False,  True, False, False, False, False, False])

In [147]:
selector.ranking_

array([12, 13,  1,  1,  1,  2,  8,  6,  9,  1, 10,  1,  4,  3, 11,  7,  5])

In [148]:
# ... Logistics Model

In [149]:
# Identify the outcome column/data
yy = mainFile4[['Net Income Score']]
yy.head()

,Net Income Score
0,1
1,3
5,3
6,3
7,3


In [150]:
yy.shape

(2982, 1)

In [151]:
XX = features1
XX.shape

(2982, 17)

In [153]:
#Create a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    XX, yy, random_state=1)

In [154]:
y_train.shape

(2236, 1)

In [155]:
X_train.shape

(2236, 17)

In [156]:
y_test.shape

(746, 1)

In [157]:
y_test.shape

(746, 1)

In [158]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [159]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.5787119856887298


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [160]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.6152815013404825


In [161]:
# ... recursive feature elimination

In [162]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

In [163]:
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False])

In [164]:
selector.ranking_

array([15, 17, 16, 14, 12,  8, 11,  6,  5,  7,  9,  3,  2,  1,  4, 13, 10])

In [ ]:
#End ... kent matthew rodgers, profitableHospital-project